# Creating a basic chat experience with kernel arguments

In this chat scenario, as the user talks back and forth with the bot, the arguments get populated with the history of the conversation. During each new run of the kernel, the arguments will be provided to the AI with content. 

In [1]:
#r "nuget: Microsoft.SemanticKernel, 1.5.0"
#!import config/Settings.cs
#!import config/Utils.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Kernel = Microsoft.SemanticKernel.Kernel;

var builder = Kernel.CreateBuilder();

// Configure AI service credentials used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

if (useAzureOpenAI)
    builder.AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey);
else
    builder.AddOpenAIChatCompletion(model, apiKey, orgId);

var kernel = builder.Build();

Installed Packages Microsoft.SemanticKernel, 1.5.0

Let's define the prompt.

In [2]:
const string skPrompt = @"
ChatBot can have a conversation with you about any topic.
It can give explicit instructions or say 'I don't know' if it does not have an answer.

{{$history}}
User: {{$userInput}}
ChatBot:";

var executionSettings = new OpenAIPromptExecutionSettings 
{
    MaxTokens = 2000,
    Temperature = 0.7,
    TopP = 0.5
};

Register your semantic function

In [3]:
var chatFunction = kernel.CreateFunctionFromPrompt(skPrompt, executionSettings);

Initialize your arguments

In [4]:
var history = "";
var arguments = new KernelArguments()
{
    ["history"] = history
};

Chat with the Bot

In [5]:
var userInput = "Hi, I'm looking for cocktail suggestions";
arguments["userInput"] = userInput;

var bot_answer = await chatFunction.InvokeAsync(kernel, arguments);

Update the history with the output and set this as the new input value for the next request

In [6]:
history += $"\nUser: {userInput}\nAI: {bot_answer}\n";
arguments["history"] = history;

Console.WriteLine(history);


User: Hi, I'm looking for cocktail suggestions
AI: Hello! I'd be happy to help you with cocktail suggestions. What kind of flavors do you enjoy? Are there any specific spirits you prefer or would like to try? Let me know so I can provide you with some tailored recommendations.



Keep Chatting!

In [7]:
Func<string, Task> Chat = async (string input) => {
    // Save new message in the arguments
    arguments["userInput"] = input;

    // Process the user message and get an answer
    var answer = await chatFunction.InvokeAsync(kernel, arguments);

    // Append the new interaction to the chat history
    var result = $"\nUser: {input}\nAI: {answer}\n";
    history += result;

    arguments["history"] = history;
    
    // Show the response
    Console.WriteLine(result);
};

In [8]:
await Chat("I would like a cocktail suggestion for breakfast. Please suggest one of them.");


User: I would like a cocktail suggestion for breakfast. Please suggest one of them.
AI: For a breakfast cocktail, you might enjoy a classic Mimosa. It's a refreshing and light cocktail made with equal parts of champagne and chilled orange juice. It's a popular choice for brunch or breakfast gatherings. Enjoy!



In [9]:
await Chat("that sounds interesting, how do you make one?");


User: that sounds interesting, how do you make one?
AI: To make a Mimosa cocktail, you will need the following ingredients:

- Champagne or sparkling wine
- Chilled orange juice

Here's how you can make a Mimosa:

1. Take a champagne flute or a wine glass and fill it halfway with chilled orange juice.
2. Slowly pour champagne or sparkling wine into the glass until it is almost full.
3. Gently stir the mixture to combine the orange juice and champagne.
4. Optionally, you can garnish the cocktail with a slice of orange or a strawberry.

Enjoy your refreshing Mimosa cocktail for breakfast or brunch!



In [10]:
await Chat("Write a C# code and print hello world?");


User: Write a C# code and print hello world?
AI: Sure! Here's a simple C# code snippet that prints "Hello, World!" to the console:

```csharp
using System;

class Program
{
    static void Main()
    {
        Console.WriteLine("Hello, World!");
    }
}
```

You can copy and paste this code into a C# development environment, compile and run it to see the output "Hello, World!" printed to the console. Let me know if you need any further assistance with C# programming!



In [11]:
await Chat("could you list some more I could try about the topic(s) mentioned?");


User: could you list some more I could try about the topic(s) mentioned?
AI: I'm glad you're interested in trying out more cocktails! Here are a few more suggestions you might enjoy:

1. Bellini: Made with Prosecco and peach puree, this cocktail is fruity and perfect for brunch.
2. Bloody Mary: A classic savory cocktail made with vodka, tomato juice, and various spices and garnishes.
3. Irish Coffee: A warm and comforting cocktail made with hot coffee, Irish whiskey, sugar, and topped with cream.
4. Aperol Spritz: A popular Italian cocktail made with Aperol, Prosecco, and a splash of soda water, served over ice with an orange slice.
5. Paloma: A refreshing tequila-based cocktail made with grapefruit soda, lime juice, and a salted rim.

Feel free to explore these cocktails and let me know if you'd like more information or recipes for any of them! Cheers!



After chatting for a while, we have built a growing history, which we are attaching to each prompt and which contains the full conversation. Let's take a look!

In [12]:
Console.WriteLine(history);


User: Hi, I'm looking for cocktail suggestions
AI: Hello! I'd be happy to help you with cocktail suggestions. What kind of flavors do you enjoy? Are there any specific spirits you prefer or would like to try? Let me know so I can provide you with some tailored recommendations.

User: I would like a cocktail suggestion for breakfast. Please suggest one of them.
AI: For a breakfast cocktail, you might enjoy a classic Mimosa. It's a refreshing and light cocktail made with equal parts of champagne and chilled orange juice. It's a popular choice for brunch or breakfast gatherings. Enjoy!

User: that sounds interesting, how do you make one?
AI: To make a Mimosa cocktail, you will need the following ingredients:

- Champagne or sparkling wine
- Chilled orange juice

Here's how you can make a Mimosa:

1. Take a champagne flute or a wine glass and fill it halfway with chilled orange juice.
2. Slowly pour champagne or sparkling wine into the glass until it is almost full.
3. Gently stir the mix